In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(0, "..")
from stamp import *
from physics_dist414pc import *
# from physics import *
from astropy.stats import sigma_clipped_stats
import re
import os
%load_ext autoreload
%autoreload 2

matplotlib.style.use('paper')

physics_table_name = "physics_table_dist414pc.txt"

table_outflow = Table.read("../outflow_hops.csv", comment='#')
table_outflow_davis = Table.read("../outflow_davisnohops.csv", comment='#')
table_hops = Table.read("../../catalogs/hops.fits")
table_davis = Table.read("../../catalogs/davis09_jets_without_hops.txt", format='ascii.tab')
outflow_def = table_outflow[table_outflow['outflow'] == '1']
outflow_may = table_outflow[table_outflow['outflow'] == '?']
outflow_hops = table_outflow[(table_outflow['outflow'] == '1') | (table_outflow['outflow'] == '?')]
outflow_davis = table_outflow_davis[(table_outflow_davis['outflow'] == '1') | (table_outflow_davis['outflow'] == '?')]

c12 = SpectralCube.read("../../cubes/mask_imfit_12co_pix_2_Tmb.fits")
c13 = SpectralCube.read("../../cubes/mask_imfit_13co_pix_2_Tmb_regrid12co.fits")
c18 = SpectralCube.read("../../cubes/mask_imfit_c18o_pix_2_Tmb.fits")






In [2]:
physics_table = Table( 
    names=["Source", "Confidence", "Lobe", "PA", "e_PA", "OA", "e_OA",
           "RAJ2000", "DEJ2000", "M_lowvel", "M_hivel", "P_lowvel",
           "P_hivel", "E_lowvel", "E_hivel",
           "vmax", "rmax", "tdyn",
           "Mdot_lowvel", "Mdot_hivel",
           "Pdot_lowvel", "Pdot_hivel",
           "Edot_lowvel", "Edot_hivel"], 
    dtype=["S10", "S1", "S10", "f8", "f8", "f8", "f8",
           "f8", "f8", "f8", "f8", "f8",
           "f8", "f8", "f8",
           "f8", "f8", "f8",
           "f8", "f8",
           "f8", "f8",
           "f8", "f8"]) 

physics_table["RAJ2000"].unit = u.deg
physics_table["DEJ2000"].unit = u.deg

physics_table["PA"].unit = u.deg
physics_table["e_PA"].unit = u.deg
physics_table["OA"].unit = u.deg
physics_table["e_OA"].unit = u.deg

physics_table["M_lowvel"].unit = u.Msun
physics_table["M_hivel"].unit = u.Msun
physics_table["P_lowvel"].unit = u.Msun/(u.km/u.s)
physics_table["P_hivel"].unit = u.Msun/(u.km/u.s)
physics_table["E_lowvel"].unit = u.erg
physics_table["E_hivel"].unit = u.erg

physics_table["vmax"].unit = u.km/u.s
physics_table["rmax"].unit = u.pc
physics_table["tdyn"].unit = u.yr

physics_table["Mdot_lowvel"].unit = u.Msun/u.yr
physics_table["Mdot_hivel"].unit = u.Msun/u.yr
physics_table["Pdot_lowvel"].unit = (u.Msun/(u.km/u.s))/u.yr
physics_table["Pdot_hivel"].unit = (u.Msun/(u.km/u.s))/u.yr
physics_table["Edot_lowvel"].unit = u.erg/u.s
physics_table["Edot_hivel"].unit = u.erg/u.s

In [17]:
ratio_list = [] 
ratio_rms_list = []
vsys_list = []
Tex_arr = np.array([])
matplotlib.style.use('paper')
# physics_table = Table.read("physics_table_hops_def.txt", format='ascii')

# outflow_table = outflow_hops
# name = "hops"
# name = "davis"
for outflow_table, name in zip([outflow_davis, outflow_hops], ["davis", "hops"]):
    for i, row in enumerate(outflow_table):
#         iwork=0
#         if i != iwork:
#             continue
    #     davis = row["davis"]
    
        n = row[name]
#         if name != "hops" or n != 158:
#             continue
            
    #     if n != 30:
    #         continue

        blue_vel, red_vel = row["blue_vel"], row["red_vel"]
        tanabe = row["tanabe"]
        note = row["notes"]
        conf = row["outflow"]
        if conf == '?':
            conf = 'M'
        elif conf == '1':
            conf = 'D'

        if name == "davis":
            davis_row = table_davis[table_davis["SMZ"] == n][0]
            coord = SkyCoord(davis_row["_RAJ2000"], davis_row["_DEJ2000"], unit=u.deg)
        elif name == "hops":
            hops_row = table_hops[table_hops["HOPS"] == n][0]
            coord = SkyCoord(hops_row["RAJ2000"], hops_row["DEJ2000"], unit=u.deg)

        plot_dir = "{}{}_plots/".format(name, n)
        os.makedirs(plot_dir, exist_ok=True)

        # Read in the hand-drawn masks for the blue/red lobes.    
        width = height = u.Quantity(glob.glob("../masks/maskroi_{}{}*.npy".format(name, n))[0].split('_')[2])
        try:
            mask_blue = np.load(glob.glob("../masks/maskroi_{}{}_*blue.npy".format(name, n))[0])
            no_blue = False
        except IndexError:
            no_blue = True
            mask_blue = False
        try:       
            mask_red = np.load(glob.glob("../masks/maskroi_{}{}_*red.npy".format(name, n))[0])
            no_red = False
        except IndexError:
            no_red = True
            mask_red = False
        try:
            mask_full = np.load(glob.glob("../masks/maskroi_{}{}_*arcmin.npy".format(name, n))[0])
        except IndexError:
            mask_full = mask_blue + mask_red

        sub12 = extract_subcube(c12, region_class=RectangleSkyRegion,
                          region_kwargs=dict(center=coord, width=width, height=height))
        sub13 = extract_subcube(c13, region_class=RectangleSkyRegion,
                            region_kwargs=dict(center=coord, width=width, height=height))

        #PARSE BLUE/RED_VEL FROM TABLE
        #     print(blue_vel, red_vel)
        if blue_vel == '-':
            fit_12 = extract_subcube(sub12, CircleSkyRegion,
                                    region_kwargs={'center':coord, 'radius':15*u.arcsec})

            blue_vel, red_vel = calc_linewings(fit_12, nsigma_vel=2.)
            blue_vel, red_vel = blue_vel.to(u.km/u.s), red_vel.to(u.km/u.s)
        #         print(blue_vel, red_vel)
        else:
            try:
                blue_vel, red_vel = float(blue_vel)*u.km/u.s, float(red_vel)*u.km/u.s
            except ValueError:
                blue_vel, red_vel = blue_vel.strip("[]").split(','), red_vel.strip("[]").split(',')
                blue_vel = [float(a) for a in blue_vel]*u.km/u.s
                red_vel = [float(a) for a in red_vel]*u.km/u.s

        print(blue_vel, red_vel)

        #Calculate RMS in emission-free channels. Only consider pixels outside outflow mask.
        rms12 = rms(sub12.with_mask(~mask_full), velocity_range=[[-2,0],[18,20]]*u.km/u.s,
                   sigma_clipped=True)
        rms13 = rms(sub13.with_mask(~mask_full), velocity_range=[[0,2], [16,18]]*u.km/u.s,
                   sigma_clipped=True)
        print(rms12, rms13)


        #Use only the pixels with a integrated intensities > SNR set below in the 
        #visually defined velocity ranges.
        outflow_mom0_snr = 5.

        try:
            slab_blue = sub12.spectral_slab(sub12.spectral_extrema[0], blue_vel)
        except:
            slab_blue = sub12.spectral_slab(blue_vel[0], blue_vel[1])
        try:
            slab_red = sub12.spectral_slab(red_vel, sub12.spectral_extrema[1])
        except:
            slab_red = sub12.spectral_slab(red_vel[0], red_vel[1])

        mask_bluelobe = mask_blue & (slab_blue.moment0() > outflow_mom0_snr*sigma_mom0(slab_blue, channel_sigma=rms12))
        mask_redlobe = mask_red & (slab_red.moment0() > outflow_mom0_snr*sigma_mom0(slab_red, channel_sigma=rms12))
        mask_outflow = mask_bluelobe + mask_redlobe

        mask12_3sig = sub12 > 3*rms12
        mask13_3sig = sub13 > 3*rms13
        mask12_5sig = sub12 > 5*rms12
        mask13_5sig = sub13 > 5*rms13
        mask12_10sig = sub12 > 10*rms12
        mask13_10sig = sub13 > 10*rms13


        #Calculate Tex using the peak temperature of the average 12co spectrum.
        Tex_peak = cube_Tex(sub12.with_mask(mask_outflow), thick=True, plot="tex.png", average=True, average_first=True)
    #     !rm plot_dir
        
        plt.savefig(plot_dir + "Tex.pdf", bbox_inches='tight')
    #     plt.show()
        plt.clf()
        Tex_arr = np.append(Tex_arr, Tex_peak)
        print(Tex_peak)
        print(name, n)

7.528555538001431 km / s 13.720211831898501 km / s
1.0302046537399292 K 0.46668294072151184 K
75.253981660121 K
davis 11
3.0 km / s 18.0 km / s
1.1467885971069336 K 0.47432810068130493 K
97.69631415917064 K
davis 17
8.0 km / s 14.0 km / s
1.0642668008804321 K 0.4479545056819916 K
108.29685904002676 K
davis 21
4.5 km / s 14.0 km / s
1.0140188932418823 K 0.4710104465484619 K
75.52404595852978 K
davis 30
4.5 km / s 11.5 km / s
0.8981247544288635 K 0.42164382338523865 K
50.16286971422975 K
davis 50
5.150106963345423 km / s 12.643003341859197 km / s
0.7722852826118469 K 0.3238241374492645 K
46.0349881809543 K
hops 10
4.656766783228081 km / s 12.841272383441655 km / s
0.7859107851982117 K 0.3360837399959564 K
53.61579705152572 K
hops 11
4.7299066016254745 km / s 12.486596600305091 km / s
0.9047529101371765 K 0.40177300572395325 K
50.5339930130931 K
hops 12
4.4 km / s 12.0 km / s
0.9900668859481812 K 0.48364394903182983 K
83.82798585086923 K
hops 44
5.8 km / s 13.0 km / s
1.2151966094970703 K

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [20]:
Tex_arr.mean()

64.45445295288806

In [28]:
sub = c12.subcube(1000,1100,1000,1100,50,55)
nH2_list = [column_density_H2(sub, Tex.value*u.K).sum().value for Tex in Tex_arr]

In [30]:
nH2_norm = column_density_H2(sub, 50.*u.K).sum().value
correction_factor = [nH2_norm / nH2 for nH2 in nH2_list]
correction_factor

[0.6938321620964661,
 0.5450814453220719,
 0.4949162928286094,
 0.6915632229355475,
 0.9971709498108875,
 1.07411949510589,
 0.9407156134705932,
 0.9907837336345342,
 0.6283540216402429,
 1.0564724789087399,
 0.541018097065216,
 0.5045830187713882,
 0.6204317812898923,
 0.5493049945583269,
 0.5352746988638671,
 0.5942536614653664,
 0.6878761215039855,
 0.6999885617892527,
 0.7501817553475885,
 0.7458097143084094,
 0.8027743978810085,
 0.8325979834655227,
 0.7911515539770863,
 0.7669673064861662,
 0.771790438066444,
 0.9133376329979717,
 1.1512084960348044,
 1.1784930731943,
 1.5190820767205937,
 0.9601555325446973,
 1.301801178326317,
 1.1243815377675022,
 1.1817793679135051,
 0.9249504929284107,
 1.0958275585381867,
 0.972130920406515,
 1.065486091301749,
 0.9338544286502453,
 0.9288246211929363,
 1.316584116322542,
 1.1679209501606154,
 1.5073653077168796,
 0.5671021256069425,
 0.5143907943195001,
 0.7619671458208007]